In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt 
import time
# 查询系统可用的 GPU
physical_devices = tf.config.experimental.list_physical_devices('GPU')
# 确保有可用的 GPU 如果没有, 则会报错
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
# 设置参数,该段务必在运行jupyter的第一段代码执行，否则会无法初始化成功
# 仅在需要时申请显存空间（程序初始运行时消耗很少的显存，随着程序的运行而动态申请显存）
tf.config.experimental.set_memory_growth(physical_devices[0], True)

#### 1. 卷积神经网络初探--解决什么问题

##### 在卷积神经网络出现之前，图像对于机器学习是一个难题，有两个原因：
- 图像需要处理的数据量太大，导致成本很高，效率很低
- 图像在数字化的过程中很难保留原有的特征，导致图像处理的准确率不高

##### 卷积神经网络将大量参数共享，实质上是降低了参数量，机器的目标是识别一张图片是什么，低像素的图片和高像素的图片实质上都可以识别一个物体是什么，不像人类，对照片的质量本身有很高的追求，且这个追求在不断延伸

![图像所需处理的数据量很大](./markdown_pics/图像所需处理的数据量很大.png)

![图片处理的传统方法](./markdown_pics/图片处理的传统方法.png)

![保留图像特征的理解](./markdown_pics/保留图像特征的理解.png)

#### 2. CNN的基本原理

- 卷积神经网络简介
- 卷积层
- 池化层
- 全连接层


![卷积神经网络介绍1](./markdown_pics/卷积神经网络介绍1.png)

下图演示了通过卷积-池化-卷积-池化-卷积-池化-卷积-池化-全连接的过程识别图像

![特斯拉识别演示卷积池化过程](./markdown_pics/特斯拉识别演示卷积池化过程.png)

##### 简单了解卷积层

![卷积动态图](./markdown_pics/卷积动态图.gif)

![卷积神经网络动态图的静态第一帧](./markdown_pics/卷积神经网络动态图的静态第一帧.jpeg)

##### 下面用numpy来演示具体的计算过程

In [30]:
# 对通道的第一个矩阵进行分割采样

input_R = np.array([
    [0,0,0],
    [0,0,1],
    [0,0,1]
])

input_G = np.array([
    [0,0,0],
    [0,1,1],
    [0,0,2]
])

input_B = np.array([
    [0,0,0],
    [0,2,0],
    [0,0,0]
])


# w0矩阵对应了三个通道的对应卷积矩阵
w0_1 = np.array([
    [1,1,-1],
    [-1,0,1],
    [-1,-1,0]
])

w0_2 = np.array([
    [-1,0,-1],
    [0,0,-1],
    [1,-1,0]
])

w0_3 = np.array([
    [0,1,0],
    [1,0,1],
    [0,-1,1]
])

# 偏置权重b0对应了第一个卷积核
b0 = 1



# w1矩阵对应了三个通道的对应卷积矩阵
w1_1 = np.array([
    [-1,-1,0],
    [-1,1,0],
    [-1,1,0]
])

w1_2 = np.array([
    [1,-1,0],
    [-1,0,-1],
    [-1,0,0]
])

w1_3 = np.array([
    [-1,0,1],
    [1,0,1],
    [0,-1,0]
])

# 偏置权重b1对应了第二个卷积核
b1 = 0

In [31]:
input_1 * w0_1

array([[0, 0, 0],
       [0, 0, 1],
       [0, 0, 0]])

In [32]:
# 对RGB通道使用卷积权重对应位置相乘并相加实现最后一个矩阵的第一个元素结果，其中b0是偏置权重
np.sum(input_R * w0_1) + np.sum(input_G * w0_2) + np.sum(input_B * w0_3) + b0

1

In [33]:
# 当生成第一个矩阵以后，开始生成第二个矩阵
np.sum(input_R * w1_1) + np.sum(input_G * w1_2) + np.sum(input_B * w1_3) + b1

-1

##### 演示完毕
$\color{orange}{至此，上面的步骤演示了动态图的一个步骤结果}$


![卷积核扫描的动图3](./markdown_pics/卷积核扫描的动图3.gif)

![卷积层的静态图演示帧数1](./markdown_pics/卷积层的静态图演示帧数1.png)

![卷积核扫描的动图2](./markdown_pics/卷积核扫描的动图2.gif)

##### 下图可以理解为25个卷积核，可以看到整个25个卷积核实际上都是由25个图像构成的，这25个小图像就可以综合来描绘一个更大的图像

![多个卷积核的近似表达理解](./markdown_pics/多个卷积核的近似表达理解.png)

![卷积层的计算公式](./markdown_pics/卷积层的计算公式.png)

![卷积层的计算公式数学表示](./markdown_pics/卷积层的计算公式数学表示.png)

##### 通过上图演示一下如何计算输出张量的高度和宽度

In [11]:
height_in = 9
height_kernel = 3
paddding = 0
stride = 1

def height_out_cal(height_in, height_kernel, paddding, stride):
    height_out = (height_in - height_kernel + 2 * paddding)/stride + 1
    return height_out

height_out_cal(height_in = height_in, height_kernel = height_kernel, paddding = paddding, stride = stride)

7.0

#### 卷积神经网络中的padding理解

问题的来源是这样的，卷积层的基本实现原理不难，但是当看到代码中的参数padding的时候感觉不是很明白，这里简单写一下自己的理解。



![卷积核的padding处理1](./markdown_pics/卷积核的padding处理1.png)

上图是简单的卷积处理方式，假设原始图片的尺寸是6x6,卷积核的大小是3x3,滑动步长为1，那么按照卷积核的滑动方式，我们可以得到feature map的尺寸为 (6−3+1)×(6−3+1),但是这样的处理方式会有两个缺点：


1.卷积后的矩阵越变越小（如果卷积层100层，每一层都缩小最终得到的将是很小的图片）

2.输入矩阵（左）边缘像素（绿阴影）只被计算过一次，而中间像素（红阴影）被卷积计算多次，意味着丢失图像角落信息。

为了解决这两个问题，就对输入图像进行padding，即**填充像素**

![卷积核的padding处理2](./markdown_pics/卷积核的padding处理2.png)

可以看到上图在原始图像的边缘用了像素填充，像素填充后的尺寸变为8×8,卷积核的大小为3×3,步长为1，经过卷积操作之后得到的feature map的尺寸是 (8−3+1)×(8−3+1)=6×6,feature map的尺寸和原始输入图像的尺寸是一样的。这样就解决了feature map尺寸越来越小的问题。同时从图中可以看出，卷积核对边缘信息的处理不止处理了一次，对边缘信息的提取更加充分了。

![TF自带的卷积层参数说明](./markdown_pics/TF自带的卷积层参数说明.png)

![TF自带的卷积层参数列表](./markdown_pics/TF自带的卷积层参数列表.png)

![TF自带的卷积层参数列表2](./markdown_pics/TF自带的卷积层参数列表2.png)

![TF自带的卷积层参数列表3](./markdown_pics/TF自带的卷积层参数列表3.png)

#### 动手实现卷积层Numpy版本

In [12]:
# x 原始张量
# w卷积核
# b 偏置
# pad 填充像素
# strides 步长 

# N 样本数
# H 高度
# W 宽度
# C 通道数

# F 输出的通道数
# HH 卷积核的高度
# WW 卷积核的宽度
# C 卷积核的通道数

def conv_numpy(x, w, b, pad, strides):
    N,H,W,C = x.shape
    F, HH, WW, C = w.shape


    # 实现填充
    X = np.pad(x, ((0,0), (pad, pad), (pad, pad),(0,0)), 'constant')

    # 计算卷积核的高和宽
    Hn = 1 + int((H + 2 * pad - HH) / strides[0])
    Wn = 1 + int((W + 2 * pad - WW) / strides[0])


    # 初始化输出的矩阵
    out = np.zeros((N, Hn, Wn, F))



    # 卷积过程，这个方法非常低效率
    for n in range(N):
        for m in range(F):
            for i in range(Hn):
                for j in range(Wn):
                    data = X[n, i * strides[0]:i * strides[0] + HH, j * strides[1]:j * strides[1] + WW, :].reshape(-1,1)
                    filt = w[m].reshape(-1,1)
                    out[n,i,j,m] = data.dot(filt) + b[m]
    return out

#### 动手实现卷积层tf2.0版本

In [13]:
def corr2d(x, w, b, pad, strides):
    N,H,W,C = tf.shape(x)
    F, HH, WW, C = tf.shape(w)

    x = tf.pad(x, ((0,0), (pad, pad), (pad, pad),(0,0)), 'constant')

        # 计算卷积核的高和宽
    Hn = 1 + int((H + 2 * pad - HH) / strides[0])
    Wn = 1 + int((W + 2 * pad - WW) / strides[0])

    Y = tf.Variable(tf.zeros(N, Hn, Wn, F), dtype = tf.float32)

    for m in range(F):
            for i in range(Hn):
                for j in range(Wn):
                    data = X[n, i * strides[0]:i * strides[0] + HH, j * strides[1]:j * strides[1] + WW, :]
                    filt = W[m,:,:,:]
                    Y[:,i,j,m].assign(tf.reduce_sum(tf.multiply(data,filt),axis = (1,2,3)) + b[m])

    return Y

### 池化层

##### 池化简单来说就是下采样，它可以大大的降低数据的维度

![池化动态图示](./markdown_pics/池化动态图示.gif)

![池化动图的文字说明](./markdown_pics/池化动图的文字说明.png)

##### 以下图为例，池化的过程就是对一个子矩阵求矩阵的元素最大值（也可以是矩阵的元素的均值），作为池化后的输出矩阵的对应位置元素值

![池化动态图示2](./markdown_pics/池化动态图示2.gif)

![池化层的计算公式](./markdown_pics/池化层的计算公式.png)

![tf自带的池化MaxPool2D](./markdown_pics/tf自带的池化MaxPool2D.png)

![tf自带的池化MaxPool2D参数说明](./markdown_pics/tf自带的池化MaxPool2D参数说明.png)

#### 动手实现池化层Numpy版本

In [3]:
def max_pool_forward_naive(x, pool_size =(2,2), strides = (1,1)):
    N,H,W,C = x.shape
    h_p,w_p = pool_size
    h_s,w_s = strides

    Hn = 1 + int((H - h_p)/h_s)
    Wn = 1 + int((W - w_p)/w_s)
    out = np.zeros((N,Hn,Wn,C))

    for i in range(Hn):
        for j in range(Wn):
            out[:,i,j,:] = np.max(x[:, i * h_s: i *h_s + h_p,j*w_s: j*w_s + w_p, :], axis = (1,2))

    return out

#### 动手实现池化层Numpy版本

In [5]:
def pool2d(x, pool_size =(2,2), strides = (1,1)):
    N,H,W,C = x.shape
    h_p,w_p = pool_size
    s_h,s_w = strides

    Y = tf.zeros((N, (H - p_h +1)// s_h, (w - p_w + 1)// s_w, C))
    Y = tf.Variable(Y)


    for i in tf.range(tf.shape(Y)[1]):
        for j in tf.range(tf.shape(Y)[2]):
            Y[:, i, j, :].assign(tf.math.reduce_max(X[:, i * s_h : i * s_h + p_h, j *s_w + p_w, :], axis = (1,2), keepdims = False))
    return Y

![全连接层的作用](./markdown_pics/全连接层的作用.png)

#### 3. CNN有哪些实际应用

- LeNet
- AlexNet
- ZFNet
- GoogleNet
- VGGNet


![卷积神经网络架构1](./markdown_pics/卷积神经网络架构1.png)

![卷积神经网络架构2](./markdown_pics/卷积神经网络架构2.png)

![卷积神经网络架构3](./markdown_pics/卷积神经网络架构3.png)

![应用_图像分类](./markdown_pics/应用_图像分类.png)

![应用_目标定位检测](./markdown_pics/应用_目标定位检测.png)

![应用_目标分割](./markdown_pics/应用_目标分割.png)

![应用_人脸识别](./markdown_pics/应用_人脸识别.png)

![应用_骨骼识别](./markdown_pics/应用_骨骼识别.png)